In [184]:
import requests
from pyvis.network import Network
import time
import argparse
import os
import math
import json

In [175]:
# address="kaspa:qqssy8x2stwk6x7trmw56m8rwfkwul70rpqxrvv789mxqz73pdny2sprry82x"
address="kaspa:qp2sp0vvrwu4s8pw0j68muu2ta5qar5mehf8ehuvljw5zsrakk5cvx4gvqz7z"
nb_cercles=4
limit=100

In [176]:
URLADDRESS="https://api.kaspa.org/addresses/"
URLTRANSAC="https://api.kaspa.org/transactions/"

responseAddress=requests.get(URLADDRESS+address+"/full-transactions")
data=responseAddress.json()

In [ ]:
addtest="kaspa:qpjunp39ssazf4rzfxxu0hd35xggfxn6lq0ls9u9q6peevzcmcv4xmv9q4njd"
testname=requests.get(URLADDRESS+addtest+"/name").json()
if "name" in testname.keys():
    print("yes")
else:
    print("no")

yes


In [204]:
def get_address_data(address):
    return requests.get(URLADDRESS+address+"/full-transactions").json()

def get_input(input):
    old_transac=input["previous_outpoint_hash"]
    old_index=int(input["previous_outpoint_index"])
    responseTransac=requests.get(URLTRANSAC+old_transac)
    dataTransac=responseTransac.json()
    for old_output in dataTransac["outputs"]:
        if old_output["index"]==old_index:
            return old_output["script_public_key_address"]
        
def get_inputs_ouputs(transac):
    inputs=[]
    outputs=[]

    if transac["inputs"]:
        for input in transac["inputs"]:
            inputs.append(get_input(input))

    if transac["outputs"]:
        for output in transac["outputs"]:
            output_add=output["script_public_key_address"]
            outputs.append(output_add)

        outputs=[output for output in outputs if output not in inputs]
    return inputs,outputs

def explore_address(relations,address,cercle,addrSeen,addrList,futurList):
    time.sleep(0.1)
    verify_name=requests.get(URLADDRESS+address+"/name").json()
    if "name" not in verify_name.keys():
        data_address=get_address_data(address)

        if address not in relations:
            relations[address]={
                "nb_transacs_in":0,
                "nb_transacs_out":0,
                "address_in":{},
                "address_out":{},
                "cercle":cercle
            }

        for transac in data_address:
            inputs,outputs=get_inputs_ouputs(transac)
            for input in inputs:
                if input!=address:
                    if input in relations[address]["address_in"]:
                        relations[address]["address_in"][input]+=1
                    else:
                        relations[address]["address_in"][input]=1
                    relations[address]["nb_transacs_in"]+=1
                    if input not in addrList and input not in futurList and input not in addrSeen:
                        futurList.append(input)

            for output in outputs:
                if output!=address:
                    if output in relations[address]["address_out"]:
                        relations[address]["address_out"][output]+=1
                    else:
                        relations[address]["address_out"][output]=1
                    relations[address]["nb_transacs_out"]+=1
                        
                    if output not in addrList and output not in futurList and output not in addrSeen:
                        futurList.append(output)

    return relations,futurList

def explore_address2(relations,address,cercle,addrSeen,addrList,futurList):
    data_address=get_address_data(address)

    for transac in data_address:
        inputs,outputs=get_inputs_ouputs(transac)
        for input in inputs:
            if input not in relations:
                relations[input]={
                    "nb_transacs_in":0,
                    "nb_transacs_out":0,
                    "address_in":{},
                    "address_out":{},
                    "cercle":cercle
                    }
                if input not in addrList and input not in futurList:
                    futurList.append(input)
            
            relations[input]["nb_transacs_out"]+=len(outputs)

            for output in outputs:
                if output in relations[input]["address_out"]:
                    relations[input]["address_out"][output]+=1
                else:
                    relations[input]["address_out"][output]=1

        for output in outputs:
            if output not in relations:
                relations[output]={
                    "nb_transacs_in":0,
                    "nb_transacs_out":0,
                    "address_in":{},
                    "address_out":{},
                    "cercle":cercle
                }
                if output not in addrList and output not in futurList:
                    futurList.append(output)

            relations[output]["nb_transacs_in"]+=len(inputs)

            for input in inputs:
                if input in relations[output]["address_in"]:
                    relations[output]["address_in"][input]+=1
                else:
                    relations[output]["address_in"][input]=1

    return relations,futurList
    
def create_vis(relations, initial_address):
    net=Network(height="900px", width="100%", bgcolor="#222222", font_color="white", directed=True)
    net.barnes_hut(gravity=-10000,central_gravity=0.3,spring_length=250,spring_strength=0.01)
    colors=["#FF0000","#FFA500","#FFFF00","#800080","#00FF00","#00FFFF","#FF00FF"]

    for addr in relations:
        if addr==initial_address:
            color="#0000FF"
            size=40
        else:
            color=colors[relations[addr]["cercle"]]
            size=25
        
        net.add_node(addr,label=addr[:10],title=addr,color=color,size=size)

    for source, data in relations.items():
        for target, count in data["address_out"].items():
            if target in relations:
                if source==initial_address:
                    edge_color="#FF0000"
                    label=str(count)
                elif target ==initial_address:
                    edge_color= "#FFFF00" 
                    label =str(count)
                else:
                    edge_color= "#666666"
                    label =str(count)

                net.add_edge(source,target,color=edge_color,value=count,title=f"{count} transacs",label=label)

    net.show("crypto_circles.html", notebook=False)

def main(initial_address,nb_cercles):
    relations={}
    addrSeen=[]
    addrList=[initial_address]

    for cercle in range(nb_cercles):
        print("cercle: ",cercle)
        futurList=[]
        for addr in addrList:
            if addr not in addrSeen:
                print("expl: ",addr)
                relations,futurList=explore_address(relations,addr,cercle,addrSeen,addrList,futurList)
                addrSeen.append(addr)
        addrList=futurList
        print(len(addrList))
    
    create_vis(relations,initial_address)
    return relations,addrSeen,futurList
address="kaspa:qp2sp0vvrwu4s8pw0j68muu2ta5qar5mehf8ehuvljw5zsrakk5cvx4gvqz7z"
relations,addrSeen,futurList=main(address,3)

cercle:  0
expl:  kaspa:qp2sp0vvrwu4s8pw0j68muu2ta5qar5mehf8ehuvljw5zsrakk5cvx4gvqz7z
12
cercle:  1
expl:  kaspa:qpjfvypc2rlxzxwh3p578xt6ud663uv358mf20wjwqv9p832t2zjj8jps2mvd
expl:  kaspa:qztrxqwp896ukrsn0yvd698hp8ww2dl0njjvts84cuymmh9hf9zg2mqh9j5mr
expl:  kaspa:qrzt54pavj9gt0vgal7a2s79w3azvvpaqvpyftwf8ehxgju38p23ql965vkey


KeyboardInterrupt: 

In [172]:
print(len(relations))

19


In [170]:
for adrr in relations:
    print(adrr)

kaspa:qp2sp0vvrwu4s8pw0j68muu2ta5qar5mehf8ehuvljw5zsrakk5cvx4gvqz7z
kaspa:qpjfvypc2rlxzxwh3p578xt6ud663uv358mf20wjwqv9p832t2zjj8jps2mvd
kaspa:qztrxqwp896ukrsn0yvd698hp8ww2dl0njjvts84cuymmh9hf9zg2mqh9j5mr
kaspa:qrzt54pavj9gt0vgal7a2s79w3azvvpaqvpyftwf8ehxgju38p23ql965vkey
kaspa:qpn3cnk57ymyhz6jc4wvleuquxc87dtyupzdeefq0lulwu0g8ryd22ww5cym7
kaspa:qzuft3vgkry6wqrqu693gxqrynvylzfu9e9p05lv5jflg72gw7qqqt0660jvc
kaspa:qzy5tjqc9fy0q66gy02tmrhngd5rwgkgj68nxpk9y67xgehy7kkljpjljst78
kaspa:qqxnx2ju886z9mkqj63m24ddcjs9f8h4w4wfhvmejje59g58pqnz6e2nh7mmg
kaspa:qqc9skh9rq636mam58587pkk4x570nzq38686x8ggtzpnnv3qf3l757k34g7n
kaspa:qz7aljpkqnxryc52jcl5v8pl8g3uz902f3ruxueylfdaqlcu8fk2g0u8vzdas
kaspa:qp4a9nuuqsfg58ee52dprx3skjwwf8am2c7aqm0sgnxrlh6g6vdf6vrnlk2fw
kaspa:qzkrllddlweq6yy2ly30chufadlnva7xkcevzkcg26zzdhufvmpawtnwwqwzp
kaspa:qzrf8zmacnzf7ymjkuglshzmyt53mghxe5kdx08u208l5xfha8vajl30fkrgj
kaspa:qrxpx7lztmrusxxqe7nhss9hy8x8ahxqya9w4s4pwqyy7y3d7ax37tejms8pd
kaspa:qr7aw56l2yp7sygs2kat4jkfkemssqlln3ps0wwt5t

In [181]:
print(relations["kaspa:qr7aw56l2yp7sygs2kat4jkfkemssqlln3ps0wwt5tjpp86cc4ptgy5nyj49q"])

{'nb_transacs_in': 0, 'nb_transacs_out': 0, 'address_in': {}, 'address_out': {}, 'cercle': 1}


In [133]:
print(address)
relations=explore_address(address)
for add in relations:
    print(add,relations[add])

kaspa:qp2sp0vvrwu4s8pw0j68muu2ta5qar5mehf8ehuvljw5zsrakk5cvx4gvqz7z
kaspa:qp2sp0vvrwu4s8pw0j68muu2ta5qar5mehf8ehuvljw5zsrakk5cvx4gvqz7z {'nb_transacs': 13, 'kaspa:qpjfvypc2rlxzxwh3p578xt6ud663uv358mf20wjwqv9p832t2zjj8jps2mvd': 1, 'kaspa:qztrxqwp896ukrsn0yvd698hp8ww2dl0njjvts84cuymmh9hf9zg2mqh9j5mr': 3, 'kaspa:qrzt54pavj9gt0vgal7a2s79w3azvvpaqvpyftwf8ehxgju38p23ql965vkey': 6, 'kaspa:qpn3cnk57ymyhz6jc4wvleuquxc87dtyupzdeefq0lulwu0g8ryd22ww5cym7': 2, 'kaspa:qqxnx2ju886z9mkqj63m24ddcjs9f8h4w4wfhvmejje59g58pqnz6e2nh7mmg': 1}
kaspa:qzuft3vgkry6wqrqu693gxqrynvylzfu9e9p05lv5jflg72gw7qqqt0660jvc {'nb_transacs': 1, 'kaspa:qp2sp0vvrwu4s8pw0j68muu2ta5qar5mehf8ehuvljw5zsrakk5cvx4gvqz7z': 1}
kaspa:qzy5tjqc9fy0q66gy02tmrhngd5rwgkgj68nxpk9y67xgehy7kkljpjljst78 {'nb_transacs': 2, 'kaspa:qp2sp0vvrwu4s8pw0j68muu2ta5qar5mehf8ehuvljw5zsrakk5cvx4gvqz7z': 1, 'kaspa:qzuft3vgkry6wqrqu693gxqrynvylzfu9e9p05lv5jflg72gw7qqqt0660jvc': 1}
kaspa:qqxnx2ju886z9mkqj63m24ddcjs9f8h4w4wfhvmejje59g58pqnz6e2nh7mmg {'nb_trans